## Clustering de Livres à partir d'une Base de Données d'Amazon
##### AGIER Olivier - CHEVALIER Arnaud

L'objectif de ce projet est de réaliser une opération de clustering sur les données brutes présentes dans une base de données du site Amazon.com . L'idée est de regrouper ces livres dans un certains nombre de clusters cohérents (livres d'un même auteur, titres similaires...), afin de pouvoir y appliquer différents algorithmes dans le cadre d'un projet d'option.

A partr des données brutes de départ, on cherche à créer une connaissance de similarité entre les livres. Ce n'est cependant pas un système de recommandation. On ne cherche pas à trier les livres par pertinence par rappport à un ou plusieurs autres, mais bien à découper les `§INSERT_NUMBER` éléments du set de base en un certain nombre de clusters de taille équivalentes.

Dans un premier temps, nous allons importer et "nettoyer" les données, nous découperons ensuite les titres et les auteurs en mots. Une pseudo distance sera ensuite définie afin d'appliquer une première clusterisation (grâce à la méthode k-means après réalisation d'une réduction de dimension) sur les mots présents dans les titres et les auteurs. Cette clusterisation permettra ensuite de réappliquer un k-means sur les vecteurs de mots correspondant aux entrées de la base de données initiale.

Enfin nous vérifierons la qualité de la clusterisation finale.

In [2]:
import pandas as pd
import pickle

## Etape 1 : Import et Nettoyage des données brutes

La base de données que nous utiliserons `amazon_livres.txt` contient beaucoup d'informations, nous allons dans un premier temps nous limiter à celles qui nous intéressent : les titres des livres et les auteurs.

L'ensemble des opérations de pré-processing pour récupérer les informations brutes initiales sont regroupées dans le dossier du même nom. Celles-ci ont été mises au point dans le cadre du projet d'option dans lequel s'inscrit ce projet. Nous ne rentrerons donc pas dans les détails ici.

In [ ]:
data = pd.read_csv("amazon_livres.txt", sep="\t", dtype=str, encoding="ISO-8859-1")

In [20]:
data = data[['ASIN', 'ISBN', 'TITLE', 'AUTHOR', 'PUBLISHER', 'PUBLICATION_DATE', 'PARUTION_ID']]
data = data[data['AUTHOR'].notnull()]
data = data[data['TITLE'].notnull()]
data.rename({'AUTHOR': 'AUTEUR', 'TITLE': 'TITRE'}, axis=1, inplace=True)

In [21]:
import pre_processing.cleaning
import pre_processing.volume_number_detection
data['author'] = pre_processing.cleaning.clean_df_column(data.AUTEUR)
data['title'] = pre_processing.cleaning.clean_df_column(data.TITRE)

In [16]:
def remove_short_words(string):
    output = str(string)
    string = string.split()
    output = output.split()
    for word in string:
        if len(word) <= 2:
            output.remove(word)
    return output

In [23]:
data.drop(columns=['ASIN',
 'ISBN',
 'TITRE',
 'AUTEUR',
 'PUBLISHER',
 'PUBLICATION_DATE',
 'PARUTION_ID'], inplace=True)

A ce stade, les données sont de la forme §INSERT_EXAMPLE . L'ensemble des éléments représente `§INSERT_NUMBER` lignes dans la table `data`.

Nous allons par la suite nous intéresser à chaque mot des différentes listes et compter son nombre de connexions avec les autres. Les mots de deux lettre ou moins ("LA", "DE", "OU", "D.", ...) auront à l'évidence un grand nombre de connexions, mais celles-ci ne sont pas intéressantes dans le cadre de notre clusterisation, nous allons donc les supprimer. 

In [46]:
dat = data.copy()

In [47]:
# enlever les mots de moins de 2 lettres
dat.author = dat.author.apply(lambda x: remove_short_words(x))
dat.title = dat.title.apply(lambda x: remove_short_words(x))

Après ce nettoyage, nous avons à disposition la table `dat` dont les éléments sont de la forme : `§INSERT_EXAMPLE`

Nous allons maintenant descendre au niveau des mots.

## Etape 2 : Correspondances entre les Mots

L'objectif de cette étape est de définir des taux de correspondance entre les différents mots présents dans la table `dat`. Pour réaliser cette opération, nous allons créer quatre dictionnaires représentant les quatre types de connexions possible entre deux mots A et B :

    - Le mot A figure dans l'Auteur d'une ligne où B est dans le Titre
    - Le mot A figure dans le Titre d'une ligne où B est dans l'Auteur
    - Les mots A et B figurent dans le même Auteur sur une ligne
    - Les mots A et B figurent dans le même Titre sur une ligne
    
Ces différentes connexions sont stockées respectivement dans les dictionnaires `name_word` `word_name` `name_name` et `word_word` . Elles ne sont pas directement regroupées afin de conserver la possibilité de les pondérer avec plusieurs coefficients en considérant que certaines sont plus importantes que d'autres. Cette pondération avait initialement été envisagée comme une possible amélioration en cas de résultat final non satisfaisant.

Les éléments des quatre dictionnaires sont de la forme : `§INSERT_EXAMPLE`

In [9]:
import numpy as np
import pandas as pd
from tqdm import tqdm

def name_word(data):
    names = dict()
    words = dict()
    freqs = dict()
    for idx, row in tqdm(data.iterrows()):
        l_auth = row.author
        l_tit = row.title
        for auth in l_auth:
            if auth not in names.keys():
                names[auth] = [idx]
            else:    
                names[auth].append(idx)
            for wd in l_tit:
                if wd not in words.keys():
                    words[wd] = [idx]
                else:
                    words[wd].append(idx)
                
                if auth not in freqs.keys():
                    freqs[auth] = {wd: 1}
                else:
                    if wd not in freqs[auth].keys():
                        freqs[auth][wd] = 1
                    else:
                        freqs[auth][wd] += 1

    return names, words, freqs


_, _, names_words = name_word(dat)

837188it [01:47, 7785.58it/s]


In [10]:
def word_name(data):
    words_names = dict()
    for idx, row in tqdm(data.iterrows()):
        l_auth = row.author
        l_tit = row.title
        for word in l_tit:
            for auth in l_auth:
                if word not in words_names.keys():
                    words_names[word] = {auth: 1}
                else:
                    if auth not in words_names[word].keys():
                        words_names[word][auth] = 1
                    else:
                        words_names[word][auth] += 1
    return words_names


words_names = word_name(dat)

837188it [01:39, 8405.46it/s]


In [11]:
def name_name(data):
    names = dict()
    for idx, row in tqdm(data.iterrows()):
        l_auth = row.author
        for auth1 in l_auth:
            for auth2 in l_auth:
                if auth1 != auth2:
                    if auth1 not in names.keys():
                        names[auth1] = {auth2: 1}
                    else:
                        if auth2 not in names[auth1].keys():
                            names[auth1][auth2] = 1
                        else:
                            names[auth1][auth2] += 1
    return names

names_names = name_name(dat)

837188it [01:17, 10818.50it/s]


In [12]:
def word_word(data):
    names = dict()
    for idx, row in tqdm(data.iterrows()):
        l_auth = row.title
        for auth1 in l_auth:
            for auth2 in l_auth:
                if auth1 != auth2:
                    if auth1 not in names.keys():
                        names[auth1] = {auth2: 1}
                    else:
                        if auth2 not in names[auth1].keys():
                            names[auth1][auth2] = 1
                        else:
                            names[auth1][auth2] += 1
    return names

words_words = word_word(dat)

837188it [01:45, 7959.21it/s]


Une fois les dictionnaires créés, ils vont être rassemblés dans un graphe global dans les cellules suivantes. Celui-ci représente l'ensemble des mots différents présents dans la table initiale `dat` et toutes leurs connexions.

Une opération de nettoyage est ensuite réalisé sur le graphe : on supprime toutes les connexions entre un mot et lui même grâce à la fonction `remove_diag`. Celles-ci sont en effet inutiles (chaque mot est unique dans le graphe) et pourraient potentiellement être gênantes dans les calculs suivants.

Le graphe est ensuite stocké en mémoire. Il contient `§INSERT_NBRE_ELTS_GRAPH` de la forme `§INSERT_EXAMPLE` .

In [13]:
def create_graph(names_names, words_words, names_words, words_names):
    graph = names_names.copy()
    for word in tqdm(words_words.keys()):
        if word not in graph.keys():
            graph[word] = words_words[word]
        else:
            for wd in words_words[word].keys():
                if wd in graph[word].keys():
                    graph[word][wd] += words_words[word][wd]
                else:
                    graph[word][wd] = words_words[word][wd]
    for word in tqdm(names_words.keys()):
        if word not in graph.keys():
            graph[word] = names_words[word]
        else:
            for wd in names_words[word].keys():
                if wd in graph[word].keys():
                    graph[word][wd] += names_words[word][wd]
                else:
                    graph[word][wd] = names_words[word][wd]
    for word in tqdm(words_names.keys()):
        if word not in graph.keys():
            graph[word] = words_names[word]
        else:
            for wd in words_names[word].keys():
                if wd in graph[word].keys():
                    graph[word][wd] += words_names[word][wd]
                else:
                    graph[word][wd] = words_names[word][wd]
    return graph

graph = create_graph(names_names, words_words, names_words, words_names)

100%|██████████| 177916/177916 [00:03<00:00, 56798.62it/s] 


In [14]:
def remove_diag(graph):
    clean_graph = dict(graph)
    for word in clean_graph.keys():
        if word in clean_graph[word].keys():
            del clean_graph[word][word]
    return clean_graph

clean_graph = remove_diag(graph)

In [15]:
import pickle
pickle.dump(file=open('graph.pickle', 'wb'), obj=clean_graph)

In [16]:
def total_weight(graph):
    total = 0
    for word in graph:
        for p in graph[word]:
            total += graph[word][p]
    return total

total = total_weight(clean_graph)

In [17]:
total

71267172

## Etape 3  : Réduction de dimension

A ce stade `clean graph` est un le graph des connexions entre un mot A et un mot B, pondéré par le nombre de connexions. Ce graph est converti en matrice d'affinité par `create_sparse_matrix`, `create_word_index` crée la table de corrspondance entre un mot et son indice dans les lignes de la matrice.

In [18]:
def create_word_index(graph):
    out = {}
    i = 0
    for word in graph.keys():
        out[word] = i
        i += 1
    return out

index = create_word_index(clean_graph)

pickle.dump(file=open('index.pickle', 'wb'), obj=index)

In [4]:
from scipy.sparse import csr_matrix
from tqdm import tqdm

def create_sparse_matrix(clean_graph, index):
    data = []
    row = []
    col = []
    for word in tqdm(clean_graph.keys()):
        for wd in clean_graph[word].keys():
            row.append(index[word])
            col.append(index[wd])
            data.append(clean_graph[word][wd])
    matrix = csr_matrix((data, (row, col)))
    return matrix
matrix = create_sparse_matrix(graph, index)

# pickle.dump(file=open('matrix.pickle', 'wb'), obj=matrix)

100%|██████████| 248804/248804 [00:18<00:00, 13447.61it/s]


### 1. IncrementalPCA

Nous avons appliqué deux algorithmes pour la réduction de dimensions, la PCA (Principal Components Analysis) et la SVD (Singular Values Decomposition). 

La PCA est un algorithme qui ne peut s'appliquer que sur des variables de type numpy.array. Cela pose un problème de mémoire car la matrice d'affinité est de taille `248804*248804` et ne tiens pas en mémoire. Pour palier ce problème, nous utilisons l'algorithme incremental PCA de sklearn qui permet de réaliser le calcul par batches.

Le calcul a pris plus de 10h, nous metttons le fichier pickle du résultats de la PCA dans le dossier.

In [ ]:
from sklearn.decomposition import IncrementalPCA
from tqdm import tqdm
import pickle

chunk_size = 100
n = matrix.shape[0]

pca = IncrementalPCA(n_components=15, batch_size=100)

for i in tqdm(range(0, n//chunk_size)):
    rows = matrix[i*chunk_size : (i+1)*chunk_size].toarray()
    pca.partial_fit(rows)

pickle.dump(file=open('pca.pickle', 'wb'), obj=pca)

In [25]:
import pickle
from sklearn.decomposition import IncrementalPCA
import pandas as pd

# pca = pickle.load(open("dac/pca.pickle", 'rb'))
# graph = pickle.load(file=open('graph.pickle', 'rb'))
index = pickle.load(file=open('../index.pickle', 'rb'))

In [16]:
transfo = []

chunk_size = 100
n = matrix.shape[0]
for i in tqdm(range(0, n//chunk_size)):
    transfo.extend(pca.transform(matrix[i*chunk_size : (i+1)*chunk_size].toarray()))


100%|██████████| 2488/2488 [15:10<00:00,  2.48it/s]


In [17]:
pickle.dump(file=open('pca_transfo.pickle', 'wb'), obj=transfo)

### 2. TruncatedSVD
Pour la SVD nous utilisons l'algorithme TruncatedSVD du module skelarn, il présente l'avantage de fonctionner avec des objets de type scipy.sparce_matrix, la matrice sparce d'affinité tient en mémoire.

In [ ]:
from sklearn.decomposition import TruncatedSVD

svd = TruncatedSVD(n_components=15, random_state=0)

svd.fit(matrix)

transfo_svd = svd.transform(matrix)

## Etape 4 : K-Means Clustering (au niveau des mots)

A présent chaque mot du lexique est représenté par sa projection dans l'espace de dimension 'n_components', le nombre de composantes choisies pour la réduction de dimension.

Dans les cellules suivantes, l'algorithme k-means avec n clusters est appliqué sur la projection des mots. Cela permet d'associer un cluster à chaque mot. Le but est ensuite de représenter les références de livres par des vecteurs de taille n et de norme 1 où chaque mot contribue pour un poids identique à la composante du cluster auquel il appartient.

In [4]:
import pickle
# transfo = pickle.load(file=open('svd_transfo.pickle', 'rb'))
transfo = pickle.load(file=open('../pca_transfo.pickle', 'rb'))

In [ ]:
from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=100, random_state=0)
kmeans.fit(transfo)


In [22]:
from collections import Counter
a = Counter(kmeans.labels_)
a

Counter({0: 210581,
         1: 1,
         2: 3,
         3: 1,
         4: 1,
         5: 18,
         6: 1,
         7: 1,
         8: 1,
         9: 7,
         10: 1,
         11: 28,
         12: 27,
         13: 1,
         14: 6,
         15: 4,
         16: 235,
         17: 2440,
         18: 1,
         19: 1,
         20: 139,
         21: 11,
         22: 1,
         23: 1,
         24: 596,
         25: 5,
         26: 1,
         27: 37,
         28: 3,
         29: 3,
         30: 8,
         31: 1,
         32: 1,
         33: 2,
         34: 33,
         35: 1,
         36: 33,
         37: 2095,
         38: 1057,
         39: 47,
         40: 1,
         41: 2,
         42: 8,
         43: 22,
         44: 10,
         45: 24,
         46: 4,
         47: 8,
         48: 1,
         49: 64,
         50: 2,
         51: 6,
         52: 149,
         53: 43,
         54: 2,
         55: 1,
         56: 5133,
         57: 7,
         58: 232,
         59: 2,
         6

In [27]:
# Fonction qui à chaque mot associe un cluster

def create_clusters_index(index, clusters):
    clusters_index = {}
    for word, row in index.items():
        if row < len(clusters):
            clusters_index[word] = clusters[row]
    return clusters_index

clusters_index = create_clusters_index(index, kmeans.labels_)

## Etape 5 : K-Means Clustering (au niveau des livres)

A ce stade, les mots sont répartis dans n clusters. Chaque mot a donc une étiquette de 0 à n-1.

Dans cette étape, les références de livres sont converties en vecteurs de taille n. Chaque mot contribue pour le même poids à la composante de son étiquette cluster. La représentation vectorielle d'un mot est ensuite normalisée. 

In [28]:
import numpy as np

def book_to_vect(clusters_index, n_clusters, book_author, book_title):
    """
    kmeans is an index of kmeans[word] = cluster_id
    """
    vect = np.zeros(n_clusters)
    for word in book_author:
        if word in clusters_index.keys():
            vect[clusters_index[word]] += 1
    for word in book_title:
        if word in clusters_index.keys():
            vect[clusters_index[word]] += 1
    norm = np.linalg.norm(vect)
    if norm:
        vect = vect/np.linalg.norm(vect)
    return vect

In [11]:
dat['coordinates'] = [[0]*100]*len(dat)

dat['coordinates'] = dat.apply(lambda x: book_to_vect(clusters_index, 100, x['author'], x['title']), axis=1)

In [124]:
pickle.dump(file=open('vectorized_db.pickle', 'wb'), obj=dat)

In [3]:
# load vectorized DB with PCA (15) and kmeans (100)

dat = pickle.load(file=open('../vectorized_db.pickle', 'rb'))

A ce stade chaque référence de livre est représentée par un vecteur de dimansion n=100.

L'ultime étape consiste à appliquer un algorithme de clustering sur les références. Le choix du nombre de clusters dépend du problème à résoudre. 

Les cellules suivantes résolvent le problème avec 10 clusters.

In [44]:
kmeans_books = KMeans(n_clusters=10, random_state=0)
kmeans_books.fit(dat.coordinates.values.tolist())

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [45]:
from collections import Counter
b = Counter(kmeans_books.labels_)
b

Counter({0: 61448,
         1: 76613,
         2: 58604,
         3: 95878,
         4: 77744,
         5: 109577,
         6: 84185,
         7: 73933,
         8: 79142,
         9: 120064})

In [46]:
dat['cluster'] = kmeans_books.labels_

In [47]:
dat[dat['cluster'] == 0]

,author,title,coordinates,cluster
7,"[lynda, curnyn]","[operation, bague, doigt]","[0.4472135954999579, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
33,"[riche, pierre]","[education, culture, dans, occident, bar]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.3779644730092272, ...",0
36,[berriot],"[julio, cortazar, enchanteur]","[0.4082482904638631, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
41,"[hubert, monteilhet]","[neropolis, roman, des, temps, neroniens]","[0.3333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
60,"[bernard, kouchner]","[que, crois]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
68,"[raymond, moody]","[vie, apres, vie]","[0.0, 0.0, 0.7559289460184544, 0.0, 0.0, 0.0, ...",0
75,"[caroline, cooney]","[photo, jenny, spring]","[0.4472135954999579, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
94,"[barbara, pease, allan, pease]","[pourquoi, les, hommes, mentent, les, femmes, ...","[0.0, 0.0, 0.0, 0.48507125007266594, 0.0, 0.0,...",0
123,"[isabel, allende]","[maison, aux, esprits]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
141,"[pierre, jean, jouve]","[scene, capitale]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0


In [48]:
final = pd.merge(data, dat, left_index=True, right_index=True)

## Etape 6 : Résultats et Vérifications

Dans cette partie, la pertinence de la vectorisation des références de livres est évaluée. Le test est effectué sur un ensemble de test qui comporte 5000 références auxquelles ont été attribuées à la main des clusters. Les références sont vectorisées à l'aide du modèle développé au dessus et différents algorithmes de clustering sont testés sur cette représentation. La performance est mesurée en terme de rappel, précision et F1_score.

In [88]:
# chargement du test_set
data = pd.read_csv("../test_set.csv", index_col=0)
data.head()

,ASIN,AUTHOR,ISBN,PARUTION_ID,PUBLICATION_DATE,PUBLISHER,TITLE,id_cluster
0,2221075943,Mark Childress,2221075943,18649.0,1995-02-28,Robert Laffont,La tête dans le carton à chapeaux,45
1,2353251684,Claire Bigard,2353251684,257114.0,2009-11-19,Editions Clair de Lune,"Méprise, Tome 2 :",47
2,2736108779,Marvin Bryan,2736108779,316861.0,1991,Sybex,Macintosh Système 7 : mode d'emploi,49
3,2744142026,Luanne Rice Francine Siéty,2744142026,272683.0,2001,France loisirs,Sarah,78
4,2702130895,J. Deaver,2702130895,257270.0,2000-01-05,Calmann-Lévy,Bone Collector,72


In [89]:
# preprocessing
import pre_processing.cleaning
import pre_processing.volume_number_detection
data['author'] = pre_processing.cleaning.clean_df_column(data.AUTHOR)
data['title'] = pre_processing.cleaning.clean_df_column(data.TITLE)

In [90]:
data.author = data.author.apply(lambda x: remove_short_words(x))
data.title = data.title.apply(lambda x: remove_short_words(x))

In [91]:
data['coordinates'] = [[0]*100]*len(data)

data['coordinates'] = data.apply(lambda x: book_to_vect(clusters_index, 100, x['author'], x['title']), axis=1)

### 1. clustering avec Kmeans

Pour le kmeans, il est supposé que le nombre k est connu a priori. 

In [93]:
new_data = data.copy()

In [ ]:
kmeans_books = KMeans(n_clusters=4743, random_state=0)
kmeans_books.fit(new_data.coordinates.values.tolist())

In [68]:
new_data['id_cluster'] = kmeans_books.labels_

In [94]:
new_data.reset_index(inplace=True)
data.reset_index(inplace=True)

In [95]:
new_data.rename({'index': 'id'}, inplace=True, axis=1)
data.rename({'index': 'id'}, inplace=True, axis=1)

In [45]:
import scoring.scoring
precision, recall, F_score = scoring.scoring.performance(new_data, data)

In [52]:
print(precision)
print(recall)
print(F_score)

0.2923433874709977
0.40384615384615385
0.33916554508748326


### 2. clustering avec DBSCAN

Cet algorithme a l'avantage de ne pas nécessiter la connaissance a priori d'une information à propos des clusters. Il ne permet pas d'obtenir de neilleures performances que Kmeans, même avec du tuning de paramètres.

In [184]:
from sklearn.cluster import DBSCAN

dbscan_books = DBSCAN(eps=0.6, min_samples=1, n_jobs=-1)
labels = dbscan_books.fit_predict(new_data.coordinates.values.tolist())

In [185]:
new_data['id_cluster'] = labels

In [186]:
import scoring.scoring
precision, recall, F_score = scoring.scoring.performance(new_data, data)
print(precision)
print(recall)
print(F_score)

0.00024055306286802005
0.6634615384615384
0.0004809317529962397


### 3. Interprétation

Lors du clustering des mots, la majorité des mots (plus de 200000) se retrouvent dans le même cluster, ils sont mal répartis par l'algorithme. 

Lors du passage à la représentation vectorielle des références de livre, un grand nombre de livres a priori différents se touve rapproché dans l'espace et un algorithmes de clustering peine à les distinguer. 

Une solution pour améliorer les performances pourrait être d'augmenter la dimension de sortie de la PCA/SVD afin de garder plus d'information descriptive des mots. Aussi certains mots apparaissent dans un grand nombre de références (parfois plus de 1% des références). Cela crée de nombreuses connexions dans la matrice d'affinité, or ces connexions ne sont pas riches en information car elles sont faites avec des mots qui sont fortment connecté. On pourrait supprimer les mots trop communs, au même titre que les mots de moins de 2 lettre.